In [1]:
import os
import numpy as np
import pandas as pd

#### Programming Note:
- I prefer to encapsulate everything within a function rather than have variables within just the notebook.
- While I understand that this takes away from some of the perks of the sandbox environment of Jupyter Notebooks, I feel that maintaining the scope of data/variables in this way prevents issues further down the road, such as
    - conflicting variables/variable names,
    - unintended changes,
    - functions confusing which data to use
    - mutability

#### Data Note
- I have chosen to undo some of the cleaning I perform in my webscraping package in order to demonstrate some common cleaning examples
- By doing this, I can simulate likely requirements when dealing with unfamiliar/third-party data sources, such as eliminating null values and cleaning up columns/rows

In [2]:
def raw_data_path() -> str:
    return '/'.join([
        os.getcwd().replace('src', 'data'),
        'season-data-raw.csv'
        
    ])

In [3]:
def load_raw() -> pd.DataFrame:
    return pd.read_csv(raw_data_path())

### Get basic overview

In [4]:
def data_overview():
    return load_raw().info()

In [5]:
data_overview()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29952 entries, 0 to 29951
Data columns (total 48 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      29952 non-null  object 
 1   Name      29939 non-null  object 
 2   Team      29952 non-null  object 
 3   Opp       29952 non-null  object 
 4   FPTS      29952 non-null  float64
 5   MP        29952 non-null  float64
 6   FPTS/MP   29952 non-null  float64
 7   Home      29952 non-null  int64  
 8   W         29952 non-null  object 
 9   W_PTS     29952 non-null  int64  
 10  L         29952 non-null  object 
 11  L_PTS     29952 non-null  int64  
 12  Injury    29952 non-null  int64  
 13  Starter   29952 non-null  int64  
 14  FG        29952 non-null  float64
 15  FGA       29952 non-null  float64
 16  FG_perc   29952 non-null  float64
 17  3P        29952 non-null  float64
 18  3PA       29952 non-null  float64
 19  3P_perc   29952 non-null  float64
 20  FT        29952 non-null  fl

**Since I scraped this data myself, there should not be very many missing values, except on the rare occassion the scraper adds a blank space for a name after injuries and then fills the stats as if they were a player who DNP that game**
</br>
</br>
Therefore, I know I need to drop these rows from the dataset. I will write a basic function to check if there are missing data or not.

In [6]:
def contains_missing_data(df) -> bool:
    return df.dropna().shape != df.shape
    
#     return f'{"No missing values anywhere" if not is_missing_values(df) else "Data is missing some values, need to fix"}'

In [7]:
contains_missing_data( load_raw() )

True

#### clean_data():
 - load the raw data 
 - do some basic cleaning, as described by comments within chain of pandas operations
 - save as a new file 'season-data-clean.csv' to se for the rest of the time

In [8]:
def clean_data_path() -> str:
    return raw_data_path().replace('raw', 'clean')

In [9]:
def clean_data() -> None:
    
#     Columns must be renamed before using this
#     Positive integers lt 255
    uint_cols: tuple[int,...] = (
        'home',
        'starter',
        'injury',
        'w_pts',
        'l_pts',
        'fg',
        'fga',
        '3p',
        '3pa',
        'ft',
        'fta',
        'pts',
        'ast',
        'orb',
        'drb',
        'trb',
        'stl',
        'blk',
        'tov',
        'pf',
        'ortg',
        'drtg'
    )
        
    rename_cols: dict[str,str] = {
        'FPTS/MP': 'fppm',
        '+/-': 'plus_minus',
#         usage, effective field goal, and true shooting percentages dont have any similar stats, unlike field goal and field goal percentage
#             can trim the '_perc' part off just these
        'USG_perc': 'usg',
        'eFG_perc': 'efg',
        'TS_perc': 'ts',
    }
    
    df: pd.DataFrame = (load_raw()
#                         Get rid of rows containing null values
                        .pipe( lambda df_: df_.loc[df_.isna().any(axis=1) == False] )
#                         Rename certain columns directly so that they can be typed in df.col_name manner
#                         and to avoid problems with subsequent step.
                        .rename( rename_cols, axis=1 )
#                         Convert all columns to lowercase, trim any possible blank spaces, and convert - to _
                        .rename( columns=lambda col_name: col_name.lower().replace(' ', '').replace('-','_') )
#                         Create new columns and correct types such that they are in optimal form
#                             new columns: total_pts, pts_diff
#                             uint8 --> integer column values < 255
                        .assign(
                            total_pts=lambda df_: df_.w_pts+df_.l_pts,
                            # will need to correct for winners/losers
                            pts_diff=lambda df_: df_.w_pts-df_.l_pts,
                            **{c:lambda df_, c=c:df_[c].astype('uint8') for c in uint_cols},
                               )
#                         Get rid of all inactive players rows
                        .pipe(lambda df_: df_.loc[df_['mp'] > 0.0])
                       )
    
#     Correct the pts_diff column to account for if team lost by that much (cannot use conditional within assign command)
    df.loc[df['team'] != df['w'], 'pts_diff'] *= -1
    
    df.to_csv(clean_data_path(), index=False)
    
    return

In [10]:
def load_clean() -> pd.DataFrame:
    return pd.read_csv(clean_data_path())

In [11]:
clean_data()

In [12]:
load_clean().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19002 entries, 0 to 19001
Data columns (total 50 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        19002 non-null  object 
 1   name        19002 non-null  object 
 2   team        19002 non-null  object 
 3   opp         19002 non-null  object 
 4   fpts        19002 non-null  float64
 5   mp          19002 non-null  float64
 6   fppm        19002 non-null  float64
 7   home        19002 non-null  int64  
 8   w           19002 non-null  object 
 9   w_pts       19002 non-null  int64  
 10  l           19002 non-null  object 
 11  l_pts       19002 non-null  int64  
 12  injury      19002 non-null  int64  
 13  starter     19002 non-null  int64  
 14  fg          19002 non-null  int64  
 15  fga         19002 non-null  int64  
 16  fg_perc     19002 non-null  float64
 17  3p          19002 non-null  int64  
 18  3pa         19002 non-null  int64  
 19  3p_perc     19002 non-nul

#### Final check that all cleaning tasks came out as desired
- No missing values in any rows
- Columns all lowercase and any special formatting taken care of
- Only specified columns for now
- Only active players

In [13]:
def final_check() -> None:
#     asserts for all type of cleaning want to do
    
    clean_df: pd.DataFrame = load_clean()
        
    assert(not contains_missing_data(clean_df))
    assert(not len([char for char in ''.join(clean_df.columns) if char.isupper() or char== '-' ]))
    assert(not len(clean_df.loc[clean_df['mp'] == 0.0]))
    
    
    return

In [14]:
final_check()